<a href="https://colab.research.google.com/github/zamirsiddiqui/Text-Classification-Using-BERT/blob/master/Text_Classification_with_BERT_using_KTRAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

KTrain Supports Tensorflow 2. We need to install it


In [0]:
!pip install pandas
!pip install tensorflow==2.0.0

In [0]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [0]:
#install ktrain on Google Colab
!pip3 install ktrain

In [0]:
# fetch the dataset using scikit-learn: For this example we are taking only 4 classess
categories = ['alt.atheism', 'soc.religion.christian',
             'comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
train_b = fetch_20newsgroups(subset='train',
   categories=categories, shuffle=True, random_state=11)
test_b = fetch_20newsgroups(subset='test',
   categories=categories, shuffle=True, random_state=11)

In [0]:
print('size of training set: %s' % (len(train_b['data'])))
print('size of validation set: %s' % (len(test_b['data'])))
print('classes: %s' % (train_b.target_names))

size of training set: 2257
size of validation set: 1502
classes: ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']


In [0]:
train_b['target_names'][:10]

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [0]:
# import ktrain and the ktrain.text modules
import ktrain
from ktrain import text

using Keras version: 2.2.4-tf


In [0]:
x_train = train_b.data
y_train = train_b.target
x_test = test_b.data
y_test = test_b.target

In [0]:
#Check first Two training example
x_train[:2]

['From: mikec@sail.LABS.TEK.COM (Micheal Cranford)\nSubject: Disney Animation\nOrganization: Tektronix, Inc., Beaverton,  OR.\nLines: 5\n\n------------------------------------\n\n  Can anyone tell me anything about the Disney Animation software package?\nNote the followup line (this is not for me but for a colleague).\n\n',
 'From: jcopelan@nyx.cs.du.edu (The One and Only)\nSubject: Re: Where are they now?\nOrganization: Salvation Army Draft Board\nLines: 31\n\nIn article <1ql0d3$5vo@dr-pepper.East.Sun.COM> geoff@East.Sun.COM writes:\n>Your posting provoked me into checking my save file for memorable\n>posts. The first I captured was by Ken Arromdee on 19 Feb 1990, on the\n>subject "Re: atheist too?". That was article #473 here; your question\n>was article #53766, which is an average of about 48 articles a day for\n>the last three years. As others have noted, the current posting rate is\n>such that my kill file is depressing large...... Among the posting I\n>saved in the early days wer

In [0]:
# Preprocess the data for BERT. use texts_from_csv instead of texts_from_array if you are reading data from csv file
# use preprocess_mode='bert'
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                       x_test=x_test, y_test=y_test,
                                                                       class_names=train_b.target_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

task: text classification
preprocessing train...
language: en


preprocessing test...
language: en


In [0]:
#check the first instance of training data
x_train[:1]

[array([[  101,  2013,  1024, ...,     0,     0,     0],
        [  101,  2013,  1024, ...,  2584,  2012,   102],
        [  101,  2013,  1024, ...,     0,     0,     0],
        ...,
        [  101,  2013,  1024, ...,  2018,  5976,   102],
        [  101,  2013,  1024, ...,  7207,  3447,   102],
        [  101,  2013,  1024, ..., 13181, 18447,   102]])]

In [0]:
# Load the BERT Model and Instantiate a Learner object
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)

Is Multi-Label? False
maxlen is 350
done.


In [13]:
# Train the model with learning rate 2e-5 and 2 epoc
# Note you can find the optimize learning rate by using learner.lr_find()
# In this poc I have just taken 2 epoc due to time constraints, you can increase the number of epoc
learner.fit_onecycle(2e-5, 2)



begin training using onecycle policy with max lr of 2e-05...
Train on 2257 samples
Epoch 1/2
2257/2257 [==============================] - 10698s 5s/sample - loss: 0.5701 - accuracy: 0.7732
Epoch 2/2
2257/2257 [==============================] - 10734s 5s/sample - loss: 0.0908 - accuracy: 0.9716


In [14]:
#use the learner.validate method to test our model against the validation set. As we can see, BERT achieves a 96% accuracy
learner.validate(val_data=(x_test, y_test), class_names=train_b.target_names)

                        precision    recall  f1-score   support

           alt.atheism       0.92      0.89      0.90       319
         comp.graphics       0.98      0.97      0.98       389
               sci.med       0.98      0.94      0.96       396
soc.religion.christian       0.91      0.98      0.94       398

              accuracy                           0.95      1502
             macro avg       0.95      0.94      0.95      1502
          weighted avg       0.95      0.95      0.95      1502



array([[283,   3,   4,  29],
       [  7, 377,   4,   1],
       [ 11,   4, 374,   7],
       [  8,   0,   1, 389]])

You can see around 97% accuracy in just two epoc. It will be close to 1 by using 5 epoc.


How to Use Our Trained BERT Model for Prediction

In [15]:
#call the learner.get_predictor method to obtain a Predictor object
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [16]:
predictor.predict(test_b.data[0:1])

['sci.med']